# Fig 1

In [1]:
using PyPlot
using Statistics
using Distributions   
using Random

# Functions  
  

In [2]:
function EIFshotEI0I(T,dt,vth,vre,vlb,tau,vT,dT,ee,ei,se,si)

    n=Int(T/dt)
    t=dt*collect(1:n)
    
    # initialize and generate random numbers
    c=0
    v=zeros(n)
    tp=zeros(Int,n)
    
    for k=1:n-1
        v[k+1]=v[k] + (dt/tau)*(dT*exp((v[k]-vT)/dT)-v[k]) + (ee-0)*se[k]+(ei-0)*si[k];
        v[k+1]>vth ? (c=c+1; v[k]=vth; v[k+1]=vre; tp[k]=1) : ()
        v[k+1]<vlb ?  v[k+1]=vlb : ()
    end
    
    r=c/T
    tp=dt*findall(tp.==1) # the spike time  
    
    return t,v,r,tp
        
end;

In [3]:
function EIFshotEI0g(T,dt,vth,vre,vlb,tau,vT,dT,ee,ei,se,si)

    n=Int(T/dt)
    t=dt*collect(1:n)
    
    # initialize and generate random numbers
    c=0
    v=zeros(n)
    tp=zeros(Int,n)
    
    for k=1:n-1
        v[k+1]=v[k] + (dt/tau)*(dT*exp((v[k]-vT)/dT)-v[k]) + (ee-v[k])*se[k] + (ei-v[k])*si[k] 
        v[k+1]>vth ? (c=c+1; v[k]=vth; v[k+1]=vre; tp[k]=1) : ()
        v[k+1]<vlb ?  v[k+1]=vlb : ()
    end
    
    r=c/T
    tp=dt*findall(tp.==1) # the spike time
    
    return t,v,r,tp

end;

---   

# Main code   

In [4]:
include("CommonCode.jl");

Neuron params:	tau=20ms, ee=60mV, ei=-10mV vT=10mV, dT=1mV
Thresh params:	vthEIF=20mV vthLIF=10
LowerB params:	vlbCond=-9.9mV vlbCurr=-15.0
Syn params:	ae,ai=1.5, -0.75mV and sig=5mV
Num params:	ThIn dv=0.0005mV & sims dt=0.01ms


In [5]:
###########################################
# Panel A - forcing term
###########################################

# get the LIF forcing t erm
vvLIF=collect(-10:0.1:vthLIF)
FLIF=-vvLIF/tau
FLIF[end]=10

# get the EIF forcing terms
vv=collect(-10:0.1:20)
FEIF=(-vv .+ dT*exp.((vv.-vT)/dT))/tau;

In [6]:
###########################################
# Panel B - distributions
###########################################

# parameters for current-based input
a=collect(-10:0.1:10)

Ae=(a.>0).*exp.(-a/ae)/ae
Ai=-(a.<0).*exp.(-a/ai)/ai

# parameters for conductance-based input

b=collect(0:0.01:1)

be=ae/ee
bi=ai/ei
betae=1/be-1
betai=1/bi-1
Be=betae*(1 .-b).^(betae-1)
Bi=betai*(1 .-b).^(betai-1)

v1=-5
aeg1=v1 .+b*(ee-v1); Aeg1=Be/(ee-v1)
aig1=v1 .+b*(ei-v1); Aig1=-Bi/(ei-v1)

v2=0;
aeg2=v2 .+b*(ee-v2); Aeg2=Be/(ee-v2)
aig2=v2 .+b*(ei-v2); Aig2=-Bi/(ei-v2)

v3=5;
aeg3=v3 .+b*(ee-v3); Aeg3=Be/(ee-v3)
aig3=v3 .+b*(ei-v3); Aig3=-Bi/(ei-v3);

In [7]:
###########################################
# panel C - timeseries examples
###########################################

Random.seed!(10);

T=500
nt=Int(T/dt);

Re,Ri=0.2,0.02

# Specific EIF PARAMS

be=ae/ee;
bi=ai/ei;
betae=1/be-1;
betai=1/bi-1;

se=rand(Bernoulli(Re*dt),nt).*rand(Beta(1,betae),nt) 
si=rand(Bernoulli(Ri*dt),nt).*rand(Beta(1,betai),nt) 

# LIF sims for current and conductance
vthLIF=10;
t,vLIFI,rLIFI,tpLIFI=EIFshotEI0I(T,dt,vthLIF,vre,vlbCurr,tau,vT,0.0,ee,ei,se,si)
t,vLIFg,rLIFg,tpLIFg=EIFshotEI0g(T,dt,vthLIF,vre,vlbCond,tau,vT,0.0,ee,ei,se,si)

# EIF sims for current and conductance
vthEIF=50
t,vEIFI,rEIFI,tpEIFI=EIFshotEI0I(T,dt,vthEIF,vre,vlbCurr,tau,vT,dT,ee,ei,se,si)
t,vEIFg,rEIFg,tpEIFg=EIFshotEI0g(T,dt,vthEIF,vre,vlbCond,tau,vT,dT,ee,ei,se,si);

In [26]:
##################################################################
# Create Figure 1
##################################################################

pygui(true); fig=figure(figsize=1.6.*(3.375,3))    

##################################################################
# Positions
##################################################################
bx,tx=0.1,0.95 # top and bottom x edges
by,ty=0.1,0.925 # top and bottom y edges
gx,gy=0.12,0.12 # gap sizes

panelfs=10
labelfs=10
tickfs=10
lw=1.0
ms=3

EIFCol="black"
LIFCol="black"
EIFCondCol="black"
EIFCurrCol="darkgrey"
LIFCondCol="black"
LIFCurrCol="darkgrey"

##################################################################
##################################################################
# PANEL 1A
##################################################################
##################################################################
xA,yA=0.1,0.525
wA,hA=0.375,0.4

axA=plt.axes([bx,yA,wA,hA]);
fig.text(xA-0.0925, 0.95, "(a)", fontsize=panelfs)
plot(vvLIF,FLIF,"-",color=LIFCol,linewidth=lw)
plot(vv,FEIF,"-",color=EIFCol,linewidth=lw)
plot([-10,20],[0,0],"k:",linewidth=lw)
xlabel("v (mv)",fontsize=labelfs); 
ylabel("f(v) (mV/ms)",fontsize=labelfs);
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
axis([-10,15,-1,2]);
axA.set_yticks([-1,0,1,2]);
axA.set_yticklabels(["-1","0","1","2"],fontsize=tickfs)
axA.text(7, 1.5, "LIF", fontsize=panelfs)
axA.text(11, 1.5, "EIF", fontsize=panelfs)

##################################################################
##################################################################
# PANEL 1B
##################################################################
##################################################################
xB,yB1,yB2=0.6,0.525,0.8
wB,hB=0.35,0.125

fig.text(xB-0.115, 0.95, "(b)", fontsize=panelfs)

axB2=plt.axes([xB,yB2,wB,hB]);
EPSPcolor="green"
IPSPcolor="red"
fill_between(aeg1, Aeg1, 0, color=EPSPcolor, alpha=0.3)
fill_between(aig1, Aig1, 0, color=IPSPcolor, alpha=0.3)
fill_between(aeg2, Aeg2, 0, color=EPSPcolor, alpha=0.3)
fill_between(aig2, Aig2, 0, color=IPSPcolor, alpha=0.3)
fill_between(aeg3, Aeg3, 0, color=EPSPcolor, alpha=0.3)
fill_between(aig3, Aig3, 0, color=IPSPcolor, alpha=0.3)
axis([-10,10,0,2.5])
ylabel("dist (cond)",fontsize=labelfs); 

fig.text(0.725, 0.715, "from v=0", fontsize=tickfs)
fig.text(0.725, 0.68, L"a_\mathrm{e}=1.5\mathrm{mV}", fontsize=tickfs)
fig.text(0.725, 0.645, L"a_\mathrm{i}=-0.75\mathrm{mV}", fontsize=tickfs)

axB2.set_yticks([0,3]);
axB2.set_yticklabels(["0","3"],fontsize=tickfs)
#yticks(rotation = 90,fontsize=tickfs)

axB1=plt.axes([xB,yB1,wB,hB]);
fill_between(a .+v1, Ae, 0, color=EPSPcolor, alpha=0.3)
fill_between(a .+v1, Ai, 0, color=IPSPcolor, alpha=0.3)
fill_between(a .+v2, Ae, 0, color=EPSPcolor, alpha=0.3)
fill_between(a .+v2, Ai, 0, color=IPSPcolor, alpha=0.3)
fill_between(a .+v3, Ae, 0, color=EPSPcolor, alpha=0.3)
fill_between(a .+v3, Ai, 0, color=IPSPcolor, alpha=0.3)
axis([-10,10,0,2.5]);
ylabel("dist (curr)",fontsize=labelfs);
xlabel("Post-synaptic potentials (mV)",fontsize=labelfs);
#xticks(fontsize=tickfs);yticks(fontsize=tickfs)
axB1.set_yticks([0,3]);
axB1.set_yticklabels(["0","3"],fontsize=tickfs)

##################################################################
##################################################################
# PANEL 1C
##################################################################
##################################################################
xC=0.1
yC1,yC2=0.275,0.1
wC,hC=0.85,0.125

LIFspike=14
EIFspike=16

fig.text(xC-0.0925, 0.425, "(c)", fontsize=panelfs)

axC1=plt.axes([bx,yC1,wC,hC]);
plot(t,vLIFI,color=LIFCurrCol,linewidth=lw)
plot(t,vLIFg,color=LIFCondCol,linewidth=lw)
plot(tpLIFI,LIFspike .+ zeros(length(tpLIFI)),"*",color=LIFCurrCol)
plot(tpLIFg,LIFspike .+ zeros(length(tpLIFg)),"*",color=LIFCondCol)
ylabel("v (mV)",fontsize=labelfs)
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
axis([0,500,0,17])
axC1.text(70, 10, "LIF", fontsize=panelfs)

axC2=plt.axes([bx,yC2,wC,hC]);
plot(t,vEIFI,color=EIFCurrCol,linewidth=lw)
plot(t,vEIFg,color=EIFCondCol,linewidth=lw)
plot(tpEIFI,EIFspike .+zeros(length(tpEIFI)),"*",color=EIFCurrCol)
plot(tpEIFg,EIFspike .+zeros(length(tpEIFg)),"*",color=EIFCondCol)
ylabel("v (mV)",fontsize=labelfs); xlabel("Time (ms)",fontsize=labelfs)
xticks(fontsize=tickfs);yticks(fontsize=tickfs)
axis([0,500,0,17])
axC2.text(70, 10, "EIF", fontsize=panelfs)

##################################################################
##################################################################
# TIDY UP ALL FIGS
##################################################################
##################################################################
for ax in [axA,axB1,axB2,axC1,axC2]
    ax.spines["top"].set_visible(false)
    ax.spines["right"].set_visible(false)
    sca(ax); yticks(fontsize=tickfs); xticks(fontsize=tickfs); 
end

savefig("Fig1.pdf")